# Fine-Tune Your Model

In [ ]:
import pathlib
import requests
import tarfile

import numpy as np
import pandas as pd
from sklearn import model_selection


def download_data(url, data_dir):
    with open(data_dir / "housing.tgz", 'wb') as f:
        response = requests.get(url)
        f.write(response.content)


def extract_data(data_dir):
    with tarfile.open(data_dir / "housing.tgz") as tgz:
        tgz.extractall(path=data_dir)


# load the data
url = "https://github.com/ageron/data/raw/main/housing.tgz"
data_dir = pathlib.Path("./sample_data")
data_dir.mkdir(parents=True, exist_ok=True)

download_data(url, data_dir)
extract_data(data_dir)
housing_df = pd.read_csv(data_dir / "housing" / "housing.csv")

# stratified sampling to match the income distribution
housing_df["income_cat"] = pd.cut(
    housing_df["median_income"],
    bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
    labels=[0, 1, 2, 3, 4]
)

train_df, test_df = model_selection.train_test_split(
    housing_df,
    test_size=0.2,
    stratify=housing_df.loc[:, "income_cat"],
    random_state=42
)

train_df.drop("income_cat", axis=1, inplace=True)
test_df.drop("income_cat", axis=1, inplace=True)

# split off the features and the target
train_features_df = train_df.drop("median_house_value", axis=1)
train_targets = train_df.loc[:, "median_house_value"]

In [ ]:
from sklearn import base, cluster, compose, impute, metrics, pipeline, preprocessing


class ClusterSimilarity(base.BaseEstimator, base.TransformerMixin):

    def __init__(self, n_clusters=10, gamma=1.0, random_state=None):
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.random_state = random_state

    def fit(self, X, y=None, sample_weight=None):
        kmeans = cluster.KMeans(
            self.n_clusters,
            n_init=10,
            random_state=self.random_state
        )
        self.kmeans_ = kmeans.fit(X, sample_weight=sample_weight)
        return self  # always return self!

    def transform(self, X):
        similarities = (
            metrics.pairwise
                   .rbf_kernel(
                       X,
                       Y=self.kmeans_.cluster_centers_,
                       gamma=self.gamma
                   )
        )
        return similarities

    def get_feature_names_out(self, names=None):
        return [f"cluster_{i:02d}_similarity" for i in range(self.n_clusters)]


def column_ratio(df):
    return df.iloc[:, 0] / df.iloc[:, 1]


def ratio_name(function_transformer, feature_names_in):
    return ["ratio"]  # feature names out


def make_ratio_pipeline():
    ratio_pipeline = (
        pipeline.make_pipeline(
            impute.SimpleImputer(strategy="median"),
            preprocessing.FunctionTransformer(column_ratio, feature_names_out=ratio_name),
            preprocessing.StandardScaler(),
            verbose=True
        ).set_output(
            transform="pandas"
        )
    )
    return ratio_pipeline


log_transform_pipeline = (
    pipeline.make_pipeline(
        impute.SimpleImputer(strategy="median"),
        preprocessing.FunctionTransformer(np.log, np.exp, feature_names_out="one-to-one"),
        preprocessing.StandardScaler()
    ).set_output(
        transform="pandas"
    )
)

cluster_similarity = (
    ClusterSimilarity(
        n_clusters=10,
        gamma=1.,
        random_state=42
    ).set_output(
        transform="pandas"
    )
)

categorical_pipeline = (
    pipeline.make_pipeline(
        impute.SimpleImputer(strategy="most_frequent"),
        preprocessing.OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    ).set_output(
        transform="pandas"
    )
)

default_numeric_pipeline = (
    pipeline.make_pipeline(
        impute.SimpleImputer(strategy="median"),
        preprocessing.StandardScaler(),
        verbose=True
    ).set_output(
        transform="pandas"
    )
)

preprocessing_pipeline = (
    compose.ColumnTransformer(
        [
            ("bedrooms", make_ratio_pipeline(), ["total_bedrooms", "total_rooms"]),
            ("rooms_per_house", make_ratio_pipeline(), ["total_rooms", "households"]),
            ("people_per_house", make_ratio_pipeline(), ["population", "households"]),
            ("log", log_transform_pipeline, ["total_bedrooms", "total_rooms", "population", "households", "median_income"]),
            ("geo", cluster_similarity, ["latitude", "longitude"]),
            ("categorical", categorical_pipeline, compose.make_column_selector(dtype_include=object)),
        ],
        n_jobs=-1,
        remainder=default_numeric_pipeline,
        verbose=True
    ).set_output(
        transform="pandas"
    )
)


In [ ]:
from sklearn import ensemble


random_forest_pipeline = (
    pipeline.Pipeline(
        [
            ("preprocessing", preprocessing_pipeline),
            ("random_forest", ensemble.RandomForestRegressor(random_state=42))
        ],
        verbose=True
    )
)

## Grid Search

In [ ]:
model_selection.GridSearchCV?

In [ ]:
param_grid = [
    {'preprocessing__geo__n_clusters': [5, 8, 10],
     'random_forest__max_features': [4, 6, 8]},
    {'preprocessing__geo__n_clusters': [10, 15],
     'random_forest__max_features': [6, 8, 10]},
]

grid_search_cv = model_selection.GridSearchCV(
    random_forest_pipeline,
    param_grid,
    cv=5,
    n_jobs=-1,
    scoring='neg_root_mean_squared_error',
    verbose=True
)

In [ ]:
grid_search_cv

In [ ]:
_ = grid_search_cv.fit(train_features_df, train_targets)

You can get the full list of hyperparameters available for tuning by looking at...

In [ ]:
(
    grid_search_cv.get_params()
                  .keys()
)

The best hyperparameter combination found:

In [ ]:
grid_search_cv.best_params_

In [ ]:
grid_search_cv.best_estimator_

Let's look at the score of each hyperparameter combination tested during the grid search:

In [ ]:
cv_results_df = pd.DataFrame(grid_search_cv.cv_results_)
cv_results_df.sort_values(by="mean_test_score", ascending=False, inplace=True)

In [ ]:
cv_results_df

## Randomized Search

In [ ]:
from scipy import stats


param_distribs = {
    'preprocessing__geo__n_clusters': stats.randint(low=3, high=50),
    'random_forest__max_features': stats.randint(low=2, high=20)
}

randomized_search_cv = model_selection.RandomizedSearchCV(
    random_forest_pipeline,
    param_distributions=param_distribs,
    n_iter=10,
    cv=3,
    scoring='neg_root_mean_squared_error',
    random_state=42,
    verbose=2
)

In [ ]:
randomized_search_cv

In [ ]:
_ = randomized_search_cv.fit(train_features_df, train_targets)

In [ ]:
cv_results_df = pd.DataFrame(randomized_search_cv.cv_results_)
cv_results_df.sort_values(by="mean_test_score", ascending=False, inplace=True)

In [ ]:
cv_results_df

**Bonus section: how to choose the sampling distribution for a hyperparameter**

* `scipy.stats.randint(a, b+1)`: for hyperparameters with _discrete_ values that range from a to b, and all values in that range seem equally likely.
* `scipy.stats.uniform(a, b)`: this is very similar, but for _continuous_ hyperparameters.
* `scipy.stats.geom(1 / scale)`: for discrete values, when you want to sample roughly in a given scale. E.g., with scale=1000 most samples will be in this ballpark, but ~10% of all samples will be <100 and ~10% will be >2300.
* `scipy.stats.expon(scale)`: this is the continuous equivalent of `geom`. Just set `scale` to the most likely value.
* `scipy.stats.loguniform(a, b)`: when you have almost no idea what the optimal hyperparameter value's scale is. If you set a=0.01 and b=100, then you're just as likely to sample a value between 0.01 and 0.1 as a value between 10 and 100.


Here are plots of the probability mass functions (for discrete variables), and probability density functions (for continuous variables) for `randint()`, `uniform()`, `geom()` and `expon()`:

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats


xs1 = np.arange(0, 7 + 1)
randint_distrib = stats.randint(0, 7 + 1).pmf(xs1)

xs2 = np.linspace(0, 7, 500)
uniform_distrib = stats.uniform(0, 7).pdf(xs2)

xs3 = np.arange(0, 7 + 1)
geom_distrib = stats.geom(0.5).pmf(xs3)

xs4 = np.linspace(0, 7, 500)
expon_distrib = stats.expon(scale=1).pdf(xs4)

plt.figure(figsize=(12, 7))

plt.subplot(2, 2, 1)
plt.bar(xs1, randint_distrib, label="scipy.randint(0, 7 + 1)")
plt.ylabel("Probability")
plt.legend()
plt.axis([-1, 8, 0, 0.2])

plt.subplot(2, 2, 2)
plt.fill_between(xs2, uniform_distrib, label="scipy.uniform(0, 7)")
plt.ylabel("PDF")
plt.legend()
plt.axis([-1, 8, 0, 0.2])

plt.subplot(2, 2, 3)
plt.bar(xs3, geom_distrib, label="scipy.geom(0.5)")
plt.xlabel("Hyperparameter value")
plt.ylabel("Probability")
plt.legend()
plt.axis([0, 7, 0, 1])

plt.subplot(2, 2, 4)
plt.fill_between(xs4, expon_distrib, label="scipy.expon(scale=1)")
plt.xlabel("Hyperparameter value")
plt.ylabel("PDF")
plt.legend()
plt.axis([0, 7, 0, 1])

plt.show()

Here are the PDF for `expon()` and `loguniform()` (left column), as well as the PDF of log(X) (right column). The right column shows the distribution of hyperparameter _scales_. You can see that `expon()` favors hyperparameters with roughly the desired scale, with a longer tail towards the smaller scales. But `loguniform()` does not favor any scale, they are all equally likely:

In [ ]:
xs1 = np.linspace(0, 7, 500)
expon_distrib = stats.expon(scale=1).pdf(xs1)

log_xs2 = np.linspace(-5, 3, 500)
log_expon_distrib = np.exp(log_xs2 - np.exp(log_xs2))

xs3 = np.linspace(0.001, 1000, 500)
loguniform_distrib = stats.loguniform(0.001, 1000).pdf(xs3)

log_xs4 = np.linspace(np.log(0.001), np.log(1000), 500)
log_loguniform_distrib = stats.uniform(np.log(0.001), np.log(1000)).pdf(log_xs4)

plt.figure(figsize=(12, 7))

plt.subplot(2, 2, 1)
plt.fill_between(xs1, expon_distrib, label="scipy.expon(scale=1)")
plt.ylabel("PDF")
plt.legend()
plt.axis([0, 7, 0, 1])

plt.subplot(2, 2, 2)
plt.fill_between(log_xs2, log_expon_distrib, label="log(X) with X ~ expon")
plt.legend()
plt.axis([-5, 3, 0, 1])

plt.subplot(2, 2, 3)
plt.fill_between(xs3, loguniform_distrib, label="scipy.loguniform(0.001, 1000)")
plt.xlabel("Hyperparameter value")
plt.ylabel("PDF")
plt.legend()
plt.axis([0.001, 1000, 0, 0.005])

plt.subplot(2, 2, 4)
plt.fill_between(log_xs4, log_loguniform_distrib, label="log(X) with X ~ loguniform")
plt.xlabel("Log of hyperparameter value")
plt.legend()
plt.axis([-8, 1, 0, 0.2])

plt.show()

## Analyze the Best Models and Their Errors

In [ ]:
randomized_search_cv.best_estimator_

In [ ]:
feature_importances = (
    randomized_search_cv.best_estimator_
                        ["random_forest"]
                        .feature_importances_
)
feature_importances

In [ ]:
feature_names = (
    randomized_search_cv.best_estimator_
                        ["preprocessing"]
                        .get_feature_names_out()
)
sorted(zip(feature_importances, feature_names), reverse=True)

## Evaluate Your System on the Test Set

In [ ]:
test_features_df = test_df.drop("median_house_value", axis=1)
test_targets = test_df.loc[:, "median_house_value"]

test_predictions = (
    randomized_search_cv.best_estimator_
                        .predict(test_features_df)
)

metrics.mean_squared_error(
    test_targets,
    test_predictions,
    squared=False
)

We can compute a 95% confidence interval for the test RMSE:

In [ ]:
confidence = 0.95
squared_errors = (test_predictions - test_targets)**2
np.sqrt(
    stats.t.interval(
        confidence,
        len(squared_errors) - 1,
        loc=squared_errors.mean(),
        scale=stats.sem(squared_errors)
    )
)

We could compute the interval manually like this:

In [ ]:
m = len(squared_errors)
mean = squared_errors.mean()
tscore = stats.t.ppf((1 + confidence) / 2, df=m - 1)
tmargin = tscore * squared_errors.std(ddof=1) / np.sqrt(m)
np.sqrt(mean - tmargin), np.sqrt(mean + tmargin)

Alternatively, we could use a z-score rather than a t-score. Since the test set is not too small, it won't make a big difference:

In [ ]:
# extra code – computes a confidence interval again using a z-score
zscore = stats.norm.ppf((1 + confidence) / 2)
zmargin = zscore * squared_errors.std(ddof=1) / np.sqrt(m)
np.sqrt(mean - zmargin), np.sqrt(mean + zmargin)